In [1]:
import torch
from torch import nn,optim
from torch.autograd import Variable
from torch.nn import init

# 1. 标准RNN

In [2]:
# ht = tanh(wih*xt + bih + whh*ht-1 + bhh)

basic_rnn = nn.RNN(input_size=20,hidden_size=50,num_layers=2)
# 输入维度20，输出维度50，两层的单向网络

In [3]:
#通过weight_ih_10来访问第一层中的wih

basic_rnn.weight_ih_l0

Parameter containing:
tensor([[ 5.0425e-02, -1.2649e-01,  1.4041e-01,  6.2836e-02,  3.3386e-02,
          1.1819e-01, -3.1185e-03,  7.6503e-02, -1.1901e-01,  4.9798e-02,
         -6.4650e-02, -1.3300e-01,  8.3401e-03, -5.0910e-02, -4.7148e-02,
          1.0019e-01, -7.8342e-02, -4.8692e-02,  3.0624e-02,  9.6287e-02],
        [-5.8611e-02,  8.6810e-02,  1.3215e-01, -2.9301e-02, -9.2425e-02,
          6.0384e-02, -5.4672e-02, -5.6219e-02, -8.4766e-03,  1.1687e-01,
          1.2545e-01,  4.5222e-02,  1.2433e-01,  7.7686e-02, -1.1716e-01,
          2.5758e-02, -7.6829e-02,  2.6284e-02, -6.9821e-02, -4.0231e-02],
        [ 2.3985e-02,  1.1238e-02,  1.1653e-01, -1.3695e-01, -9.6351e-02,
         -7.6548e-02, -9.7093e-02,  8.6296e-02,  1.5471e-02,  4.9487e-02,
          1.0732e-01, -8.7509e-02,  4.6828e-02, -1.6464e-02, -7.0952e-02,
         -5.7222e-02,  7.3083e-02,  8.6464e-02,  1.1467e-01,  4.7909e-02],
        [-2.6739e-02, -6.2148e-02,  2.9273e-02,  1.1097e-01,  3.5280e-02,
          8.7

In [4]:
# 通过weight_hh_10来访问第一层中的whh

basic_rnn.weight_hh_l0

Parameter containing:
tensor([[ 0.0241, -0.0237,  0.0459,  ...,  0.0993, -0.0868,  0.0893],
        [ 0.0356, -0.0579,  0.1193,  ..., -0.0649,  0.0382, -0.0557],
        [-0.1077,  0.0845, -0.1009,  ..., -0.0323,  0.0338, -0.0301],
        ...,
        [-0.0159,  0.0316,  0.1043,  ...,  0.1199, -0.0642,  0.0120],
        [-0.1222,  0.0491, -0.0845,  ..., -0.0439,  0.0270, -0.1280],
        [-0.0832, -0.0842, -0.1036,  ...,  0.1210, -0.0547, -0.0432]],
       requires_grad=True)

In [5]:
# 通过bias_ih_10来访问第一层中的bih

basic_rnn.bias_ih_l0

Parameter containing:
tensor([ 0.0443,  0.0472,  0.0873,  0.1395,  0.1197, -0.0130,  0.1006,  0.1324,
        -0.0479, -0.1128, -0.1154, -0.0433, -0.0138,  0.0042, -0.0392, -0.1322,
        -0.0474, -0.0135, -0.0242, -0.0452, -0.1019,  0.0417, -0.0749, -0.1211,
        -0.0346,  0.0587,  0.0719, -0.1062,  0.0076, -0.0857, -0.1402,  0.1312,
        -0.1294,  0.1063, -0.0189,  0.0395, -0.0662,  0.0068, -0.1307,  0.0583,
         0.0514,  0.0856, -0.1245, -0.1019,  0.0838, -0.1273,  0.0310, -0.1028,
         0.0521, -0.0923], requires_grad=True)

In [6]:
#随机初始化输入和隐藏状态,输入是一个长为100，批量为32，维度为20的张量

toy_input = Variable(torch.randn(100,32,20))
h_0 = Variable(torch.randn(2,32,50)) #layer*direction,batch,hidden_size

In [7]:
#将输入状态和隐藏状态传入网络

toy_output,h_n = basic_rnn(toy_input,h_0)
toy_output.size(),h_n.size()

(torch.Size([100, 32, 50]), torch.Size([2, 32, 50]))

如果在传入网络的时候不特别注明隐藏状态h_0，那么初始的隐藏状态默认参数全是0

# 2. LSTM

LSTM中间比标准RNN多了三个线性变换，多的三个线性变换的权重拼在一起，所以一共是4倍，同理偏置也是4倍。 换句话说，LSTM里面做了4个类似标准RNN所做的运算，所以参数个数是标准RNN的4倍。

In [8]:
lstm = nn.LSTM(input_size=20,hidden_size=50,num_layers=2)
lstm_out,(h_n,c_n) = lstm(toy_input)
lstm_out.size(),h_n.size(),c_n.size()

(torch.Size([100, 32, 50]), torch.Size([2, 32, 50]), torch.Size([2, 32, 50]))

# 3.GRU

GRU的隐藏状态数量为标准RNN的3倍；
网络的隐藏状态不是 $h_0 和 c_0$，而是只有 $h_0$；
其余部分和LSTM相同；

# 4. 单步版本

RNNCell,LSTMCell,GRUCell
输入不再是一个序列，而是序列中的一步